In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import squarify
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('transaction_preprocess.csv', encoding = "ISO-8859-1")
# Convert InvoiceDate from object to datetime format
df['createdAt'] = pd.to_datetime(df['createdAt'])

In [3]:
snapshot_date = df['createdAt'].max() + timedelta(days=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256334 entries, 0 to 256333
Data columns (total 33 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Unnamed: 0              256334 non-null  int64         
 1   payerAccountNumber      256294 non-null  float64       
 2   payerIdTransformed      256334 non-null  int64         
 3   payeeIdTransformed      256334 non-null  int64         
 4   originalAmount          256334 non-null  float64       
 5   createdAt               256334 non-null  datetime64[ns]
 6   transaction_date        256334 non-null  object        
 7   transaction_time        256334 non-null  object        
 8   issuer_bank             256334 non-null  int64         
 9   ip_address              256334 non-null  int64         
 10  browser                 256334 non-null  int64         
 11  funding_method          256334 non-null  int64         
 12  secure_3ds              256334

In [4]:
df['payerAccNo'] = df['payerAccountNumber']
data_process = df.groupby(['payerIdTransformed','payerAccNo'], as_index=False).agg({
        'createdAt': lambda x: (snapshot_date - x.max()).days,
        'payerAccountNumber': 'count',
        'originalAmount':'sum',
        'payeeIdTransformed':'count',
        'status':'sum',
        'type':'sum',
        'currency':'sum',
        'amount_high':'sum',
        'bank_approved':'sum',
        'bank_err':'sum',
        'bank_invalid_card':'sum',
        'bank_insufficient_fund':'sum',
        'bank_do_not_honor':'sum',
        'bank_lost_card':'sum',
        'channel_master':'sum',
        'channel_visa':'sum',
        'channel_amex':'sum',
        'initialized_api':'sum',
        'initialized_session':'sum',
        'initialized_plugin':'sum',
        'initialized_ext_api':'sum',
        'initialized_ipg':'sum'
    
})

In [5]:
data_process.rename(columns={'createdAt': 'recency',
                         'payerAccountNumber': 'frequency',
                         'originalAmount': 'total_value',
                             'payeeIdTransformed':'merchant_frequency',
                            'status':'success_count',
                           'type':'one_time_count',
                           'currency':'lkr_trans',
                           'bank_approved': 'tot_bank_approves',
                            'bank_err': 'tot_bank_err',
                            'bank_invalid_card':'tot_invalid_card',
                            'bank_insufficient_fund':'tot_insufficient_fund',
                            'bank_do_not_honor':'tot_bank_do_not_honor',
                            'bank_lost_card':'tot_bank_lost_card',
                            'amount_high':'tot_amount_high',
                            'channel_master':'tot_channel_master',
                            'channel_visa':'tot_channel_visa',
                            'channel_amex':'tot_channel_amex',
                            'initialized_api':'tot_api',
                            'initialized_session':'tot_session',
                            'initialized_plugin':'tot_plugin',
                            'initialized_ext_api':'tot_ext_api',
                            'initialized_ipg':'tot_ipg'}, inplace=True)
data_process['unique_id'] = data_process.payerIdTransformed.astype(str) + '_' + data_process.payerAccNo.astype(str)

In [6]:
def success_tans(x):
   names = {
       'tot_success_amt': x[x['status'] ==1]['originalAmount'].sum()}

   return pd.Series(names, index=['tot_success_amt'])

df1 = df.groupby(['payerIdTransformed','payerAccNo'],as_index=False).apply(success_tans)
df1['unique_id'] = df1.payerIdTransformed.astype(str) + '_' + df1.payerAccNo.astype(str)
df1.head()
    

,payerIdTransformed,payerAccNo,tot_success_amt,unique_id
0,170,4.216892e+09,211.0,170_4216892051.0
1,170,4.691767e+09,0.0,170_4691767287.0
2,171,4.157394e+09,10.0,171_4157394003.0
3,171,4.216892e+09,530.0,171_4216892051.0
4,172,4.511936e+09,1.0,172_4511936075.0


In [7]:
data_process = pd.merge(data_process,df1[['unique_id','tot_success_amt']],on='unique_id', how='left')

In [8]:
data_process['success_percent']= data_process['success_count'].div(data_process['frequency']).mul(100).round()
data_process['avg_success_amt'] = data_process['tot_success_amt'].div(data_process['success_count']).round(2).replace(np.nan, 0)
data_process['avg_failed_amt'] = (data_process['total_value']-data_process['tot_success_amt']).div((data_process['frequency'] - data_process['success_count'])).round(2).replace(np.nan, 0)


In [9]:
df2 = (data_process.groupby(['payerAccNo'])['payerIdTransformed'].apply(lambda x: len(x.unique())))
df2=df2.to_frame().reset_index()
df2 = df2.rename(columns= {'payerIdTransformed': 'same_card_diff_user_count'})
data_process = pd.merge(data_process,df2[['payerAccNo','same_card_diff_user_count']],on='payerAccNo', how='left')
data_process.head()

,payerIdTransformed,payerAccNo,recency,frequency,total_value,merchant_frequency,success_count,one_time_count,lkr_trans,tot_amount_high,...,tot_session,tot_plugin,tot_ext_api,tot_ipg,unique_id,tot_success_amt,success_percent,avg_success_amt,avg_failed_amt,same_card_diff_user_count
0,170,4.216892e+09,124,34,213542.0,34,18,34,34,4,...,0,0,0,0,170_4216892051.0,211.0,53.0,11.72,13333.19,4
1,170,4.691767e+09,256,6,611.0,6,0,6,6,0,...,0,0,0,0,170_4691767287.0,0.0,0.0,0.00,101.83,1
2,171,4.157394e+09,261,3,30.0,3,1,3,3,0,...,0,0,0,0,171_4157394003.0,10.0,33.0,10.00,10.00,3
3,171,4.216892e+09,235,6,561.0,6,3,6,6,0,...,0,0,0,0,171_4216892051.0,530.0,50.0,176.67,10.33,4
4,172,4.511936e+09,258,12,1316.0,12,1,12,12,0,...,0,0,0,0,172_4511936075.0,1.0,8.0,1.00,119.55,1


In [10]:
df3 = (data_process.groupby(['payerIdTransformed'])['payerAccNo'].apply(lambda x: len(x.unique()))) 
df3=df3.to_frame().reset_index()
df3 = df3.rename(columns= {'payerAccNo': 'user_dist_crd_cnt'})
data_process = pd.merge(data_process,df3[['payerIdTransformed','user_dist_crd_cnt']],on='payerIdTransformed', how='left')

#data_process.DataFrame({'list':sf.values})

In [11]:
def suspicious_card(x):
   if x['same_card_diff_user_count'] > 1 :
      return 1
   return 0


data_process['suspicious_card'] = data_process.apply (lambda x: suspicious_card(x), axis=1)
data_process.head()

,payerIdTransformed,payerAccNo,recency,frequency,total_value,merchant_frequency,success_count,one_time_count,lkr_trans,tot_amount_high,...,tot_ext_api,tot_ipg,unique_id,tot_success_amt,success_percent,avg_success_amt,avg_failed_amt,same_card_diff_user_count,user_dist_crd_cnt,suspicious_card
0,170,4.216892e+09,124,34,213542.0,34,18,34,34,4,...,0,0,170_4216892051.0,211.0,53.0,11.72,13333.19,4,2,1
1,170,4.691767e+09,256,6,611.0,6,0,6,6,0,...,0,0,170_4691767287.0,0.0,0.0,0.00,101.83,1,2,0
2,171,4.157394e+09,261,3,30.0,3,1,3,3,0,...,0,0,171_4157394003.0,10.0,33.0,10.00,10.00,3,2,1
3,171,4.216892e+09,235,6,561.0,6,3,6,6,0,...,0,0,171_4216892051.0,530.0,50.0,176.67,10.33,4,2,1
4,172,4.511936e+09,258,12,1316.0,12,1,12,12,0,...,0,0,172_4511936075.0,1.0,8.0,1.00,119.55,1,1,0


In [12]:
def suspicious_user(x):
    if x['user_dist_crd_cnt'] > 2 :
      return 1
    return 0


data_process['suspicious_user_crd_count'] = data_process.apply (lambda x: suspicious_user(x), axis=1)
data_process.info()
    

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41276 entries, 0 to 41275
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   payerIdTransformed         41276 non-null  int64  
 1   payerAccNo                 41276 non-null  float64
 2   recency                    41276 non-null  int64  
 3   frequency                  41276 non-null  int64  
 4   total_value                41276 non-null  float64
 5   merchant_frequency         41276 non-null  int64  
 6   success_count              41276 non-null  int64  
 7   one_time_count             41276 non-null  int64  
 8   lkr_trans                  41276 non-null  int64  
 9   tot_amount_high            41276 non-null  int64  
 10  tot_bank_approves          41276 non-null  int64  
 11  tot_bank_err               41276 non-null  int64  
 12  tot_invalid_card           41276 non-null  int64  
 13  tot_insufficient_fund      41276 non-null  int

In [13]:
transaction_count_threshold=10
def suspicious_failure_rate(x):
    if ((x['frequency']> transaction_count_threshold) and ((100 - x['success_percent']) > 50)):
      return 1
    return 0
data_process['suspicious_failure_rate'] = data_process.apply (lambda x: suspicious_failure_rate(x), axis=1)
data_process.head()

,payerIdTransformed,payerAccNo,recency,frequency,total_value,merchant_frequency,success_count,one_time_count,lkr_trans,tot_amount_high,...,unique_id,tot_success_amt,success_percent,avg_success_amt,avg_failed_amt,same_card_diff_user_count,user_dist_crd_cnt,suspicious_card,suspicious_user_crd_count,suspicious_failure_rate
0,170,4.216892e+09,124,34,213542.0,34,18,34,34,4,...,170_4216892051.0,211.0,53.0,11.72,13333.19,4,2,1,0,0
1,170,4.691767e+09,256,6,611.0,6,0,6,6,0,...,170_4691767287.0,0.0,0.0,0.00,101.83,1,2,0,0,0
2,171,4.157394e+09,261,3,30.0,3,1,3,3,0,...,171_4157394003.0,10.0,33.0,10.00,10.00,3,2,1,0,0
3,171,4.216892e+09,235,6,561.0,6,3,6,6,0,...,171_4216892051.0,530.0,50.0,176.67,10.33,4,2,1,0,0
4,172,4.511936e+09,258,12,1316.0,12,1,12,12,0,...,172_4511936075.0,1.0,8.0,1.00,119.55,1,1,0,0,1


In [14]:
def recurring_suspicious_banker_response(x):
    tot_error=x[['tot_bank_err','tot_invalid_card','tot_insufficient_fund','tot_bank_do_not_honor','tot_bank_lost_card']].sum()
    if(tot_error>0 and x['frequency']):
        if(((x['frequency']-x['one_time_count'])>0) and ((round((x['tot_insufficient_fund']/tot_error)*100,2) <50))):
         return 1
    
    return 0
    
data_process['rec_sus_banker_res'] = data_process.apply (lambda x: recurring_suspicious_banker_response(x), axis=1)
data_process.head(5) 


,payerIdTransformed,payerAccNo,recency,frequency,total_value,merchant_frequency,success_count,one_time_count,lkr_trans,tot_amount_high,...,tot_success_amt,success_percent,avg_success_amt,avg_failed_amt,same_card_diff_user_count,user_dist_crd_cnt,suspicious_card,suspicious_user_crd_count,suspicious_failure_rate,rec_sus_banker_res
0,170,4.216892e+09,124,34,213542.0,34,18,34,34,4,...,211.0,53.0,11.72,13333.19,4,2,1,0,0,0
1,170,4.691767e+09,256,6,611.0,6,0,6,6,0,...,0.0,0.0,0.00,101.83,1,2,0,0,0,0
2,171,4.157394e+09,261,3,30.0,3,1,3,3,0,...,10.0,33.0,10.00,10.00,3,2,1,0,0,0
3,171,4.216892e+09,235,6,561.0,6,3,6,6,0,...,530.0,50.0,176.67,10.33,4,2,1,0,0,0
4,172,4.511936e+09,258,12,1316.0,12,1,12,12,0,...,1.0,8.0,1.00,119.55,1,1,0,0,1,0


In [15]:
def one_time_suspicious_banker_response(x):
    tot_error=x[['tot_bank_err','tot_invalid_card','tot_insufficient_fund','tot_bank_do_not_honor','tot_bank_lost_card']].sum()

    if(((x['one_time_count'])>0) and ((100-x['success_percent'])>50)):
     return 1
    
    return 0
    
data_process['one_tm_sus_banker_res'] = data_process.apply (lambda x: one_time_suspicious_banker_response(x), axis=1)
data_process.head(5) 

,payerIdTransformed,payerAccNo,recency,frequency,total_value,merchant_frequency,success_count,one_time_count,lkr_trans,tot_amount_high,...,success_percent,avg_success_amt,avg_failed_amt,same_card_diff_user_count,user_dist_crd_cnt,suspicious_card,suspicious_user_crd_count,suspicious_failure_rate,rec_sus_banker_res,one_tm_sus_banker_res
0,170,4.216892e+09,124,34,213542.0,34,18,34,34,4,...,53.0,11.72,13333.19,4,2,1,0,0,0,0
1,170,4.691767e+09,256,6,611.0,6,0,6,6,0,...,0.0,0.00,101.83,1,2,0,0,0,0,1
2,171,4.157394e+09,261,3,30.0,3,1,3,3,0,...,33.0,10.00,10.00,3,2,1,0,0,0,1
3,171,4.216892e+09,235,6,561.0,6,3,6,6,0,...,50.0,176.67,10.33,4,2,1,0,0,0,0
4,172,4.511936e+09,258,12,1316.0,12,1,12,12,0,...,8.0,1.00,119.55,1,1,0,0,1,0,1


In [57]:
def getSuspiciousAmount(x):
    x['time_diff']=0
    x.head()
    for i in range(x.shape[0]-1):
     #print(x['createdAt'])
     #x['time_diff'][i+1] = (x['createdAt'][i+1]-x['createdAt'][i]).time()
     #print(x['time_diff'][i+1])


df4 = df.groupby(['payerIdTransformed']).apply(lambda _df: _df.sort_values(by=['createdAt']))
df4.head()
#getSuspiciousAmount(df4)


IndentationError: expected an indented block (<ipython-input-57-a998cd3c85da>, line 10)

In [17]:
df['tran_date'] = df['transaction_date']
df4 = df.groupby(['payerIdTransformed','transaction_date']).apply(lambda _df: _df.sort_values(by=['createdAt']))
df4['daily_transaction_count']=df4.agg({'tran_date':'count'})
#df4['transactions_count_per_day'] =  df4.groupby(df4.index.transaction_date).count()

df4.head()

Unnamed: 0  payerAccountNumber  \
payerIdTransformed transaction_date                                         
170                2020-06-22       8143         8143        4.216892e+09   
                   2020-07-01       14244       14244        4.216892e+09   
                   2020-08-09       0               0        4.216892e+09   
                                    1               1        4.216892e+09   
                                    2               2        4.216892e+09   

                                           payerIdTransformed  \
payerIdTransformed transaction_date                             
170                2020-06-22       8143                  170   
                   2020-07-01       14244                 170   
                   2020-08-09       0                     170   
                                    1                     170   
                                    2                     170   

                                           payeeIdTransformed  originalAmount  \
payerIdTransformed transaction_date                                             
170                2020-06-22       8143                   39          2070.0   
                   2020-07-01       14244                  39          5175.0   
                   2020-08-09       0                       1            55.0   
                                    1                       1            10.0   
                                    2                       1            10.0   

                                                    createdAt  \
payerIdTransformed transaction_date                             
170                2020-06-22       8143  2020-06-22 09:05:02   
                   2020-07-01       14244 2020-07-01 14:23:32   
                   2020-08-09       0     2020-08-09 07:50:35   
                                    1     2020-08-09 07:54:25   
                                    2     2020-08-09 07:56:44   

                                          transaction_date transaction_time  \
payerIdTransformed transaction_date                                           
170                2020-06-22       8143        2020-06-22         09:05:02   
                   2020-07-01       14244       2020-07-01         14:23:32   
                   2020-08-09       0           2020-08-09         07:50:35   
                                    1           2020-08-09         07:54:25   
                                    2           2020-08-09         07:56:44   

                                           issuer_bank  ip_address  ...  \
payerIdTransformed transaction_date                                 ...   
170                2020-06-22       8143           395       29910  ...   
                   2020-07-01       14244          395       29910  ...   
                   2020-08-09       0              395       29910  ...   
                                    1              395       29910  ...   
                                    2              395       29910  ...   

                                           channel_amex  initialized_api  \
payerIdTransformed transaction_date                                        
170                2020-06-22       8143              0                1   
                   2020-07-01       14244             0                1   
                   2020-08-09       0                 0                1   
                                    1                 0                1   
                                    2                 0                1   

                                           initialized_session  \
payerIdTransformed transaction_date                              
170                2020-06-22       8143                     0   
                   2020-07-01       14244                    0   
                   2020-08-09       0                        0   
                                    1                        0   
              

In [ ]:
def chek3ds():
    